## 3a

In [1]:
class NDArray:
    def __init__(self, shape, default_value=0):
        self.shape = shape
        self.data = self._create_ndarray(shape, default_value)
    
    def _create_ndarray(self, shape, default_value, level=0):
        if level == len(shape) - 1:
            return [default_value for _ in range(shape[level])]
        else:
            return [self._create_ndarray(shape, default_value, level + 1) for _ in range(shape[level])]
    
    def set_item(self, index, value):
        self._set_item(self.data, index, value)
    
    def _set_item(self, data, index, value):
        if len(index) == 1:
            data[index[0]] = value
        else:
            self._set_item(data[index[0]], index[1:], value)
    
    def get_item(self, index):
        return self._get_item(self.data, index)
    
    def _get_item(self, data, index):
        if len(index) == 1:
            return data[index[0]]
        else:
            return self._get_item(data[index[0]], index[1:])
    
    def __repr__(self):
        return str(self.data)

# e.g
ndarray = NDArray(shape=(2, 3, 4)) 
ndarray.set_item((1, 2, 3), 10)  
print(ndarray.get_item((1, 2, 3)))  
print(ndarray) 

10


## b

In [2]:
class NDArray:
    def __init__(self, shape, default_value=0):
        self.shape = shape
        self.data = self._create_ndarray(shape, default_value)
    
    def _create_ndarray(self, shape, default_value):
        if len(shape) == 1:
            return [default_value] * shape[0]
        else:
            return [self._create_ndarray(shape[1:], default_value) for _ in range(shape[0])]
    
    def __getitem__(self, index):
        element = self.data
        if not isinstance(index, tuple):
            index = (index,)
        for idx in index:
            element = element[idx]
        return element

ndarray = NDArray(shape=(2, 3, 4))
print(ndarray[1, 2, 3])  

0


## c

In [1]:
class NDArrayIterator:
    def __init__(self, ndarray, index_pattern):
        self.ndarray = ndarray
        self.index_pattern = index_pattern
        self.shape = self._determine_shape(index_pattern)
        self.current_index = [0] * len(self.shape)
        self.total_steps = self._calculate_total_steps()
        self.current_step = 0

    def _determine_shape(self, index_pattern):
        shape = []
        for i, index in enumerate(index_pattern):
            if isinstance(index, slice):
                start, stop, step = index.indices(self.ndarray.shape[i])
                shape.append(len(range(start, stop, step)))
            elif isinstance(index, int):
                shape.append(1)
        return shape

    def _calculate_total_steps(self):
        total = 1
        for size in self.shape:
            total *= size
        return total

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_step >= self.total_steps:
            raise StopIteration
        result = self.ndarray[tuple(self._calculate_current_index())]
        self._increment_index()
        self.current_step += 1
        return result

    def _calculate_current_index(self):
        index = []
        for i, part in enumerate(self.index_pattern):
            if isinstance(part, slice):
                index.append(slice(*part.indices(self.ndarray.shape[i]))[self.current_index[i]])
            elif isinstance(part, int):
                index.append(part)
        return index

    def _increment_index(self):
        for i in range(len(self.current_index) - 1, -1, -1):
            if self.current_index[i] < self.shape[i] - 1:
                self.current_index[i] += 1
                break
            else:
                self.current_index[i] = 0

    def __getitem__(self, step):
        if step >= self.total_steps:
            raise IndexError("Step out of range")
        temp_index = self.current_index[:]
        self.current_step = step
        for i in range(len(temp_index)):
            temp_index[i] = step % self.shape[i]
            step //= self.shape[i]
        return self.ndarray[tuple(self._calculate_current_index(temp_index))]

    def _calculate_current_index(self, temp_index=None):
        if temp_index is None:
            temp_index = self.current_index
 